In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
train = pd.read_csv("data/train.csv")

In [6]:
train.head(1000)

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,3424175,0,11053717,49.000,W,5033,269.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3206179,0,5150800,57.950,W,4527,376.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3410455,0,10713596,115.950,W,2772,512.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3058578,0,1611950,100.000,H,14290,512.0,150.0,visa,226.0,...,chrome 62.0,24.0,1920x1080,match_status:2,T,F,T,T,desktop,Windows
4,3242131,0,6111087,95.000,W,12839,321.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,3118918,1,2613680,34.556,C,9633,296.0,185.0,visa,138.0,...,chrome 63.0,NaN,NaN,NaN,F,F,T,T,desktop,Windows
996,3243137,1,6129370,232.013,C,5009,296.0,185.0,visa,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,3141913,0,3198968,87.950,W,10057,225.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
998,3285689,0,7392074,47.950,W,7858,321.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
train.describe()


,TransactionID,isFraud,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2,...,id_17,id_18,id_19,id_20,id_21,id_22,id_24,id_25,id_26,id_32
count,4.429050e+05,442905.000000,4.429050e+05,442905.000000,442905.000000,436201.000000,441724.000000,439730.000000,393811.000000,393811.000000,...,104324.000000,33788.000000,104289.000000,104244.000000,3865.000000,3870.000000,3552.000000,3842.000000,3865.000000,58109.000000
mean,3.282059e+06,0.034989,7.366857e+06,135.273690,9901.266082,362.542367,153.188550,199.276736,290.654596,86.799658,...,189.429604,14.233278,353.493475,403.862774,370.234670,16.025581,12.851351,328.556221,149.187581,26.498202
std,1.705119e+05,0.183753,4.617568e+06,232.955213,4901.960786,157.784513,11.314301,41.232396,101.745146,2.684822,...,30.360205,1.561150,141.183369,152.298443,198.839627,6.947584,2.417639,97.420034,32.180887,3.733750
min,2.987002e+06,0.000000,8.646900e+04,0.251000,1001.000000,100.000000,100.000000,100.000000,100.000000,10.000000,...,100.000000,10.000000,100.000000,100.000000,100.000000,10.000000,11.000000,100.000000,100.000000,0.000000
25%,3.134224e+06,0.000000,3.021834e+06,43.744000,6019.000000,214.000000,150.000000,166.000000,204.000000,87.000000,...,166.000000,13.000000,266.000000,256.000000,252.000000,14.000000,11.000000,321.000000,119.000000,24.000000
50%,3.282155e+06,0.000000,7.301614e+06,68.950000,9680.000000,361.000000,150.000000,226.000000,299.000000,87.000000,...,166.000000,15.000000,341.000000,472.000000,252.000000,14.000000,11.000000,321.000000,149.000000,24.000000
75%,3.429574e+06,0.000000,1.123641e+07,125.000000,14184.000000,512.000000,150.000000,226.000000,330.000000,87.000000,...,225.000000,15.000000,427.000000,533.000000,500.000000,14.000000,15.000000,367.750000,169.000000,32.000000
max,3.577539e+06,1.000000,1.581113e+07,6085.230000,18396.000000,600.000000,231.000000,237.000000,540.000000,102.000000,...,229.000000,29.000000,671.000000,660.000000,854.000000,44.000000,26.000000,548.000000,216.000000,32.000000


In [5]:
train.corr()

C:\Users\way2s\AppData\Local\Temp\ipykernel_20096\2189804198.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  train.corr()


,TransactionID,isFraud,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2,...,id_17,id_18,id_19,id_20,id_21,id_22,id_24,id_25,id_26,id_32
TransactionID,1.000000,0.012338,0.998279,0.012531,0.009450,-0.019345,-0.009854,-0.025593,-0.000960,0.058037,...,0.223102,0.113953,-0.017802,0.083882,-0.023017,0.045451,-0.040828,0.019278,0.013935,-0.066744
isFraud,0.012338,1.000000,0.011304,0.011252,-0.014852,0.003554,0.153826,-0.034211,0.005840,-0.028248,...,0.150009,0.049143,-0.042442,0.061787,0.054894,0.110234,0.010113,0.053891,0.109625,0.068088
TransactionDT,0.998279,0.011304,1.000000,0.012430,0.009997,-0.018671,-0.010957,-0.025865,-0.000733,0.052748,...,0.215022,0.112339,-0.014493,0.081765,-0.019644,0.043564,-0.033928,0.018362,0.017519,-0.071737
TransactionAmt,0.012531,0.011252,0.012430,1.000000,-0.005598,0.016083,-0.112855,0.003697,-0.007367,0.029649,...,-0.334255,-0.096155,0.076254,-0.104815,-0.028527,-0.105114,0.103822,0.028652,0.025517,-0.043172
card1,0.009450,-0.014852,0.009997,-0.005598,1.000000,0.005205,0.001289,-0.093504,0.020588,-0.000172,...,0.017495,0.007944,0.021554,0.018466,0.045642,0.008236,0.072549,-0.013929,0.074362,-0.006123
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
id_22,0.045451,0.110234,0.043564,-0.105114,0.008236,-0.029711,0.666539,-0.181656,0.057258,-0.350926,...,0.622696,0.125243,0.062995,0.039689,0.081544,1.000000,0.009276,0.194014,0.289774,-0.047439
id_24,-0.040828,0.010113,-0.033928,0.103822,0.072549,0.000817,0.006689,-0.021828,-0.034710,-0.027411,...,-0.003893,0.021249,0.247940,-0.188381,0.220646,0.009276,1.000000,-0.033239,0.088769,-0.217872
id_25,0.019278,0.053891,0.018362,0.028652,-0.013929,-0.003238,0.149482,-0.041637,0.013948,0.021186,...,0.165969,0.109584,-0.023667,-0.019435,-0.153426,0.194014,-0.033239,1.000000,0.034907,0.039329
id_26,0.013935,0.109625,0.017519,0.025517,0.074362,-0.002425,0.242057,-0.097655,0.062177,-0.050903,...,0.094549,-0.038653,0.112868,0.053525,0.057514,0.289774,0.088769,0.034907,1.000000,-0.072881
